# Análisis Exploratorio de Datos (EDA)
## Classification Project: Predicting Default Risk in Commercial Loans

**Objetivo:** Construir un modelo de clasificación multiclase para predecir el nivel de riesgo de impago de clientes que solicitan un préstamo comercial.

**Clases objetivo:**
- 0: Bajo riesgo
- 1: Riesgo medio  
- 2: Alto riesgo

**Dataset:** 20,000 instancias con 35 características divididas en:
- Información Financiera (15 features)
- Historial de Pagos (10 features)
- Datos Demográficos y Comportamiento (10 features)

## 1. Setup y Carga de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Configurar paths
sys.path.append('../src')

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

# Importar funciones personalizadas
from data.loader import (
    load_training_data, 
    load_test_data, 
    get_feature_info, 
    separate_features_target,
    encode_target_labels,
    check_missing_values
)

print("Librerías importadas correctamente")

In [ ]:
# Cargar datos de entrenamiento y prueba
train_data = load_training_data('../data/raw/datos_entrenamiento_riesgo.csv')
test_data = load_test_data('../data/raw/datos_prueba_riesgo.csv')

print(f"Datos de entrenamiento: {train_data.shape}")
print(f"Datos de prueba: {test_data.shape}")

# Mostrar información básica
print("\n--- Información del Dataset de Entrenamiento ---")
print(train_data.info())

print("\n--- Primeras 5 filas ---")
display(train_data.head())

## 2. Análisis de la Variable Objetivo

In [ ]:
# Análisis de distribución de clases
target_counts = train_data['nivel_riesgo'].value_counts()
target_percentages = train_data['nivel_riesgo'].value_counts(normalize=True) * 100

print("Distribución de clases:")
print(f"Bajo:  {target_counts['Bajo']:,} ({target_percentages['Bajo']:.1f}%)")
print(f"Medio: {target_counts['Medio']:,} ({target_percentages['Medio']:.1f}%)")
print(f"Alto:  {target_counts['Alto']:,} ({target_percentages['Alto']:.1f}%)")

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
target_counts.plot(kind='bar', ax=ax1, color=['green', 'orange', 'red'])
ax1.set_title('Distribución de Clases - Conteo')
ax1.set_xlabel('Nivel de Riesgo')
ax1.set_ylabel('Número de Instancias')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pastel
ax2.pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', 
        colors=['green', 'orange', 'red'], startangle=90)
ax2.set_title('Distribución de Clases - Porcentaje')

plt.tight_layout()
plt.show()

# Verificar balance de clases
print(f"\nRatio de desbalance:")
print(f"Clase mayoritaria / Clase minoritaria: {target_counts.max() / target_counts.min():.2f}")

## 3. Análisis de Valores Faltantes

In [ ]:
# Verificar valores faltantes
missing_train = check_missing_values(train_data)
missing_test = check_missing_values(test_data)

print("Valores faltantes en datos de entrenamiento:")
if len(missing_train) > 0:
    display(missing_train)
else:
    print("No hay valores faltantes en el dataset de entrenamiento")

print("\nValores faltantes en datos de prueba:")
if len(missing_test) > 0:
    display(missing_test)
else:
    print("No hay valores faltantes en el dataset de prueba")

# Verificar tipos de datos
print("\nTipos de datos:")
print(train_data.dtypes.value_counts())

# Identificar features categóricas y numéricas
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()

if 'nivel_riesgo' in categorical_features:
    categorical_features.remove('nivel_riesgo')
if 'nivel_riesgo' in numerical_features:
    numerical_features.remove('nivel_riesgo')

print(f"\nFeatures categóricas ({len(categorical_features)}): {categorical_features}")
print(f"Features numéricas ({len(numerical_features)}): {len(numerical_features)} features")

## 4. Estadísticas Descriptivas

In [ ]:
# Estadísticas descriptivas para features numéricas
print("Estadísticas descriptivas - Features numéricas:")
display(train_data[numerical_features].describe())

# Verificar distribuciones extremas
print("\nFeatures con alta variabilidad (CV > 1.0):")
numerical_stats = train_data[numerical_features].describe()
cv_values = numerical_stats.loc['std'] / numerical_stats.loc['mean']
high_cv_features = cv_values[cv_values > 1.0].sort_values(ascending=False)
display(high_cv_features.head(10))

print(f"\nRango de valores por feature:")
for feature in numerical_features[:10]:  # Mostrar solo los primeros 10
    min_val = train_data[feature].min()
    max_val = train_data[feature].max()
    range_val = max_val - min_val
    print(f"{feature:30}: [{min_val:10.2f}, {max_val:10.2f}] (rango: {range_val:10.2f})")

## 5. Análisis de Correlaciones

In [ ]:
# Matriz de correlación
correlation_matrix = train_data[numerical_features].corr()

# Encontrar correlaciones altas entre features (>0.8)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                correlation_matrix.iloc[i, j]
            ))

print("Pares de features con alta correlación (>0.8):")
for feat1, feat2, corr in high_corr_pairs:
    print(f"{feat1:30} - {feat2:30}: {corr:.3f}")

# Visualizar matriz de correlación (solo una muestra de features importantes)
plt.figure(figsize=(16, 12))
sample_features = numerical_features[:15]  # Primeras 15 features para visualización
sample_corr = train_data[sample_features].corr()

sns.heatmap(sample_corr, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación - Muestra de Features Numéricas')
plt.tight_layout()
plt.show()

## 6. Análisis de Features por Categoría

In [ ]:
# Obtener información de features por categoría
feature_info = get_feature_info()

print("CATEGORÍAS DE FEATURES:")
print("="*50)

for category, features in feature_info.items():
    if category != 'target':
        print(f"\n{category.upper()} ({len(features)} features):")
        for i, feature in enumerate(features, 1):
            print(f"  {i:2d}. {feature}")

# Análisis de distribuciones por categoría de riesgo
def plot_feature_distributions(features_list, title, n_cols=3):
    n_features = len(features_list)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, feature in enumerate(features_list[:min(9, len(features_list))]):  # Máximo 9 plots
        if feature in train_data.columns:
            for risk_level in ['Bajo', 'Medio', 'Alto']:
                data_subset = train_data[train_data['nivel_riesgo'] == risk_level][feature]
                axes[i].hist(data_subset, alpha=0.6, label=f'Riesgo {risk_level}', bins=30)
            
            axes[i].set_title(f'{feature}')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)
    
    # Ocultar ejes vacíos
    for i in range(len(features_list), len(axes)):
        axes[i].set_visible(False)
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualizar distribuciones de features financieras
plot_feature_distributions(feature_info['financial'][:9], "Distribuciones - Features Financieras por Nivel de Riesgo")

In [ ]:
# Visualizar distribuciones de features de historial de pagos
plot_feature_distributions(feature_info['payment_history'][:9], "Distribuciones - Features de Historial de Pagos por Nivel de Riesgo")

## 7. Análisis de Outliers

In [ ]:
# Detectar outliers usando el método IQR
from data.loader import detect_outliers_iqr

# Analizar outliers en features seleccionadas
selected_features_outliers = ['deuda_total', 'monto_solicitado', 'patrimonio_neto', 
                             'puntuacion_credito_bureau', 'tasa_interes']

print("ANÁLISIS DE OUTLIERS (Método IQR):")
print("="*50)

outlier_summary = {}
for feature in selected_features_outliers:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(train_data, feature)
    outlier_percentage = (len(outliers) / len(train_data)) * 100
    
    outlier_summary[feature] = {
        'count': len(outliers),
        'percentage': outlier_percentage,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }
    
    print(f"\n{feature}:")
    print(f"  Outliers: {len(outliers):,} ({outlier_percentage:.2f}%)")
    print(f"  Límites: [{lower_bound:.2f}, {upper_bound:.2f}]")

# Visualización de outliers con boxplots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, feature in enumerate(selected_features_outliers):
    train_data.boxplot(column=feature, by='nivel_riesgo', ax=axes[i])
    axes[i].set_title(f'Boxplot: {feature}')
    axes[i].set_xlabel('Nivel de Riesgo')

# Ocultar el último subplot si no se usa
axes[-1].set_visible(False)

plt.suptitle('Análisis de Outliers por Nivel de Riesgo', fontsize=16)
plt.tight_layout()
plt.show()

## 8. Conclusiones del EDA y Próximos Pasos

### Hallazgos Principales:

1. **Distribución de Clases:** 
   - Verificar si hay desbalance significativo entre las clases
   - Considerar estrategias de balanceo si es necesario

2. **Calidad de Datos:**
   - Verificar presencia de valores faltantes
   - Identificar features con alta variabilidad

3. **Correlaciones:**
   - Identificar features altamente correlacionadas para posible eliminación
   - Detectar relaciones importantes con la variable objetivo

4. **Outliers:**
   - Evaluar impacto de outliers en el rendimiento del modelo
   - Decidir estrategia de tratamiento (eliminación vs transformación)

### Próximos Pasos:

1. **Preprocesamiento:**
   - Normalización/estandarización de features numéricas
   - Codificación de variables categóricas
   - Tratamiento de outliers

2. **Feature Engineering:**
   - Creación de nuevas features
   - Selección de features más relevantes
   - Aplicación de PCA si es necesario

3. **Modelado:**
   - Implementación de algoritmos desde cero
   - Validación cruzada
   - Optimización de hiperparámetros

4. **Evaluación:**
   - Métricas específicas para clasificación multiclase
   - Análisis de matriz de confusión
   - Consideración de costos de clasificación errónea